In [4]:
import pandas as pd
import pandas_ta as ta
import datetime
import numpy as np
import pickle

In [25]:
def connectToBinance():
  from binance import Client
  return Client('3FqLXkx5UOAX10B5lVRhmDDFap7uFY8evLeOqFqnwVfmkZh9IY1tr1to8oBwNdRG',
                'UUR5wLkgzqXEMQbRzkwxJlwD9iChahjLyYNanzawTCDX1jIxRUzpxVtZvcSyjAi1')

def getCurrentTicker(client, symbol):
  price = client.get_ticker(symbol=symbol, windowSize=['1h'])
  closeTime = pd.to_datetime(datetime.datetime.fromtimestamp(price['closeTime']/1000, tz=datetime.timezone.utc))
  highPrice = np.float32(price['highPrice'])
  lowPrice = np.float32(price['lowPrice'])
  closePrice = np.float32(price['lastPrice'])
  openPrice = np.float32(price['openPrice'])
  print(price)
  return pd.DataFrame({'high':highPrice,'low':lowPrice, 'close' :closePrice}, index=[closeTime.strftime("%Y-%m-%d")])

def reteiveOldData(candle,symbol):
  f = 'OldData/last48'+symbol+'.pkl'
  with open(f, 'rb') as file:
    last48Hours = pickle.load(file)
  pd.concat([last48Hours, candle]).iloc[1:].to_pickle(f)
  newLast48Hours = pd.concat([last48Hours, candle]).iloc[1:]
  print('newLast48Hours ' +symbol + ' created (new len : '+ str(len(newLast48Hours)) + ')')
  return newLast48Hours

def save48Hours(data, symbol):
  f = 'OldData/last48'+symbol+'.pkl'
  data.iloc[1:].to_pickle(f)
  
def getIndicators(data):
  strat = ta.Strategy(name='Strategie',
                     description='MACD, RSI',
                     ta=[
                      {'kind':'rsi', 'length':7},
                      {'kind':'rsi', 'length':14},
                      {'kind':'sma', 'length':7},
                      {'kind':'sma', 'length':25},
                      {'kind':'macd', 'fast':12 , 'slow':26},
                      {'kind':'obv'},
                     ])

  data.ta.strategy(strat)
  print(data.iloc[-1])
  return data.iloc[-1]

def getPrediction(indicators, symbol):
  filename='Models/'+symbol+'_model.pkl'
  with open(filename, 'rb') as file:
    model = pickle.load(file)
  return model.predict(np.array(indicators).reshape(1, -1))
  
def sellPapier(amount, file):
  print(f"Une vente de {amount}BTC à été effectuée.\n\n")
  
def buyPapier(amount, file):
  print(f"Un achat de {amount}BTC à été effectué.\n\n")

def waitPapier(file):
  print("Aucune action à été effectué.\n\n")

def paperTrade(prediction, amount):
  file = open('log.txt', 'a')
  file.write(f"{datetime.datetime.now()} - La predicton {'UP' if prediction == 2 else ('DOWN' if prediction == 0 else 'NO_MOVEMENT')} a été émise.\n")
  
  
  if(prediction == 0):
    sellPapier(amount,file)
  if(prediction == 2):
    buyPapier(amount,file)
  if(prediction == 1):
    waitPapier(amount,file)
  
   
  

client = connectToBinance()
candle = getCurrentTicker(client, 'ADAUSDT')
data = reteiveOldData(candle, 'ADAUSDT')
save48Hours(data,'ADAUSDT')
indicators = getIndicators(data)
prediction = getPrediction(indicators, 'ADAUSDT')


# 0:Down
# 1:no mouvement greater then +/-50$
# 2:Up
# import random
# prediction = random.randint(0, 2) ##Evidement à effacer
# amount = getAmount(client)
# paperTrade(prediction, amount)





# if(prediction == 0):
# elif(prediction==1):
# else:
  


BinanceAPIException: APIError(code=-1104): Not all sent parameters were read; read '1' parameter(s) but was sent '2'.

In [24]:
#Reset last34Days

def getNamesOfDataFile():
  from os import listdir
  return [f for f in listdir('Datasets') if f != '.DS_Store']


for f in getNamesOfDataFile():
  fileName = 'Datasets/'+ f
  df = pd.read_csv(fileName, header=1, parse_dates=True, infer_datetime_format=True,index_col='date')
  df.dropna(inplace=True, how='any')
  df.drop(columns=['unix', 'Volume USDT', 'tradecount','symbol'], inplace=True)
  last48Hours = df[-48:]

  last48Hours.to_pickle('OldData/last48'+f.split('_')[1]+'.pkl')
  
  
  


In [9]:
prediction

NameError: name 'prediction' is not defined

In [35]:
#Log prediction
import datetime
import random

prediction = random.randint(0, 2)
f = open('log.txt', 'a')
f.write(f"{datetime.datetime.now()} - La predicton {'UP' if prediction == 2 else ('DOWN' if prediction == 0 else 'NO_MOVEMENT')} a été émise.\n")
f.close()